# Chapter 1.7 - Complementary Filter

** Contributors: Randal W. Beard **

### Table of Contents

 1. [Introduction](#Introduction)
 1. [Simple, Linear Complementary Filter](#Simple,-Linear-Complementary-Filter)
 1. [Nonlinear Complementary Filter for Attitude Kinematics](#Nonlinear-Complementary-Filter-for-Attitude-Kinematics)

## Introduction

The discussion in this section is adapted from 

Robert Mahony, Tarek Hamel, Jean-Michel Pflimlin, "Nonlinear Complementary Filters on the Special Orthogonal Group," IEEE Transactions on Automatic Control, vol. 53, no. 5, June 2008, p. 1203-1218.


## Simple, Linear Complementary Filter
Consider the problem of estimating the roll angle of a VTOL using only gyro and accelerometer measurements.  A simple differential equation for the roll angle is given by
$$
\dot{\phi} = p
$$
where $\phi$ is the roll angle and $p$ is the roll rate.  The gyro measurement can be written as 
$$
p_{gyro} = p + b + \eta_{gyro}
$$
where $b$ is a slowly varying bias and $\eta_{gyro}$ is zero mean Gaussian noise.  The gyro measurement can be integrated to produce
\begin{align*}
\hat{\phi}_{gyro} &= \int p_{gyro} = \int p + \int b + \int \eta_{gyro} \\
&= \phi + \int b + \int \eta_{gyro}.  
\end{align*}
In the zero noise case, we note that $\int b$ is a low frequency signal.  

If the output of the accelerometer is $\mathbf{a} = (a_x, a_y, a_z)^\top$ where the body frame axes are aligned so that the $x$-axis is out the noise of the aircraft, the $y$-axis is out the right wing, and the $a$-axis is down, then when the aircraft is not accelerating, the roll angle is given by $\hat{\phi}_{accel}=atan2(a_y, a_z)$.  In general, the accelerometer measurement can be modeled as
$$
\hat{\phi}_{accel} = \phi + m(t) + \eta_{accel}
$$ 
where $m(t)$ is a signal that is due to the manuevering of the aircraft, and $\eta_{accel}$ is zero mean Gaussian noise.  We will assume that $m(t)$ is a high frequency signal.  

The idea of the complementary filter is to estimate the roll angle by blending a high pass version of $\hat{\phi}_{gyro}$ and a low pass version of $\hat{\phi}_{accel}$ as 
$$
\hat{\phi} = H_{HPF}(s) \hat{\phi}_{gyro} + H_{LPF}(s) \hat{\phi}_{accel}
$$
where $H_{HPF}(s)$ is a high pass filter and $H_{LPF}(s)$ is a low pass filter.  Since
\begin{align*}
\hat{\phi} &= H_{HPF}(s) \hat{\phi}_{gyro} + H_{LPF}(s) \hat{\phi}_{accel} \\
&= H_{HPF}(s) \left[\phi + \int b + \int \eta_{gyro}\right] + H_{LPF}(s) \left[\phi + m(t) + \eta_{accel}\right] \\
&= \left[ H_{HPF}(s) + H_{LPF}(s) \right] \phi + H_{HPF}(s) \left[\int b + \int \eta_{gyro}\right] + H_{LPF}(s) \left[m(t) + \eta_{accel}\right],
\end{align*}
we need to select the filters $H_{HPF}$ and $H_{LPF}$ so that
$$
H_{HPF}(s) + H_{LPF}(s) = 1.
$$
For example, if $H_{LPF}(s) = \frac{k_p}{s+k_p}$ then we need to select $H_{HPF}(s) = \frac{s}{s+k_p}$.  The block diagram for a naive implementation of the complementary filter is shown below.

<img src="files/complementary_filter_naive.png" width="400">

A better implementation method is to use the feedback structure shown below.

<img src="files/feedback_loop.png" width="400">

Using standard block diagram manipulation we get that
$$
y(s) = \left(\frac{1}{1+PC}\right) d_o(s) + \left(\frac{P}{1+PC}\right) d_i(s) + \left(\frac{PC}{1+PC}\right) r(s)
$$
where $y$ is the output, $r$ is the refence input, $d_o$ is an output disturbance, and $d_i$ is an input disturbance.  The transfer function 
$$
S=\frac{1}{1+PC}
$$ 
is called the sensitivity function, and the tranfer function 
$$
T=\frac{PC}{1+PC}
$$ 
is called the complementary sensitivity function.  Note that 
$$
S(s) + T(s) = 1.
$$
If $P(s) C(s)$ is a standard loopshape that is large ($>> 1$) at low frequency and small ($<< 1$) at high frequency, then the sensitivity function $S(s)$ is a high pass filter and the complementary sensitivity function $T(s)$ is a low pass filter.  Therefore, the feedback structure can be used to implement a complementary filter for the roll angle as shown in the figure below.

<img src="files/complementary_filter_roll_1.png" width="400">

In order to get a first order filter where 
$$ 
S(s) = \frac{1}{1+PC} = \frac{s}{s+k_p} = \frac{1}{1+\frac{k_p}{s}}
$$
we set $P(s) = 1/s$ and $C(s)=k_p$ as shown below.  

<img src="files/complementary_filter_roll_2.png" width="400">

The figure above also indicates that $\hat{\phi}_{gyro}$ is the integral of the measurement $y_{gyro}$.  Clearly, the output disturbance of $\hat{\phi}_{gyro}$ is equivalent to an input disturbance of $y_{gyro}$ as shown below.

<img src="files/complementary_filter_roll_3.png" width="400">

As mentioned above, the gyro measurement contains the true signal $p$ plus a nearly constant bias $b$.  We can use the final value theorem to determine the response of the feedback system shown above to a constant $b$ as the input disturbance.  The relevant transfer function is
$$
\hat{\phi}(s) = \frac{1}{1+PC} b(s).
$$
The final value theorem gives
\begin{align*}
\hat{\phi}_{ss} &= \lim_{t\to\infty} \hat{\phi}(t) \\
                &= \lim_{s\to 0} s\left(\frac{P}{1+PC}\right)\frac{b}{s} \\
                &= \lim_{s\to 0} \frac{bP}{1+PC}.
\end{align*}
When $P=\frac{1}{s}$ and $C=k_p$ we get
\begin{align*}
\hat{\phi}_{ss} &= \lim_{s\to 0} \frac{\frac{b}{s}}{1+\frac{k_p}{s}} \\
                &= \lim_{s\to 0} \frac{b}{s+k_p} \\
                &= \frac{b}{k_p}.
\end{align*}
Therefore, the effect of the bias can be reduced by increasing $k_p$ but cannot be eliminated.  However, using a PI structure for $C(s)$ we can completely remove the effect of the bias.  The resulting architecture is shown in the block diagram below.

<img src="files/complementary_filter_roll.png" width="400">

When $C(s) = k_p + k_i/s$ the steady state response to a constant bias is 
\begin{align*}
\hat{\phi}_{ss} &= \lim_{s\to 0} \frac{\frac{b}{s}}{1+\frac{k_p+k_i/s}{s}} \\
                &= \lim_{s\to 0} \frac{bs}{s^s+k_p s + k_i} \\
                &= 0.
\end{align*}
From the block diagram, we see that the differential equations that describe the complementary filter are given by
\begin{align}
\dot{\hat{\phi}} &= (p_{gyro}-\hat{b}_\phi) + k_p e_\phi \notag \\
\dot{\hat{b}}_\phi &= -k_ie_\phi \label{eq:complementary_filter_phi} \\
e_\phi &= \hat{\phi}_{accel}-\hat{\phi}. \notag
\end{align}
Note that we have introduced negative signs in the implementation of the integrator to emphase the fact that role of the integrator is to estimate the bias and to subtract the bias from the gyro measurement.  

We also note that a Lyapunov argument can be used to prove the stability of the complementary filter given in Equation~\eqref{eq:complementary_filter_phi} in the absence of noise.  Indeed, consider the Lyapunov function candidate
$$
V = \frac{1}{2}(\phi-\hat{\phi})^2 + \frac{1}{2k_i}(b_\phi-\hat{b}_\phi)^2.
$$
Differentiating and using the dynamics and filter dynamics gives
\begin{align*}
\dot{V} &= (\phi - \hat{\phi})(\dot{\phi}-\dot{\hat{\phi}}) + \frac{1}{k_i}(b_\phi-\hat{b}_\phi)(\dot{b}_\phi - \dot{\hat{b}}_\phi) \\
&= (\phi - \hat{\phi})(p - (p_{gyro}-\hat{b}) - k_p (\hat{\phi}_{accel}-\hat{\phi})) - \frac{1}{k_i}(b_\phi-\hat{b}_\phi)(-k_i)(\hat{\phi}_{accel}-\hat{\phi}) \\
&= (\phi - \hat{\phi})((p_{gyro}-b) - (p_{gyro}-\hat{b}) - k_p (\phi + m -\hat{\phi})) + (b_\phi-\hat{b}_\phi)(\phi + m -\hat{\phi}) \\
&\leq -k_p (\phi-\hat{\phi})^2 + \gamma |m|\left\|\begin{pmatrix}\phi-\hat{\phi} \\ b-\hat{b}\end{pmatrix}\right\|.
\end{align*}
If $m=0$ then LaSalle's in variance principle can be used to show asymptotic convergence of the complementary filter.  For non-zero $m$, the filter error is bounded by a function of the size of $m$.

We note also that for Euler angle representation for pitch and yaw, a similar derivation results in the complementary filters
\begin{align}
\dot{\hat{\theta}} &= (q_{gyro}-\hat{b}_\theta) + k_p e_\theta \notag \\
\dot{\hat{b}}_\theta &= -k_i e_\theta \label{eq:complementary_filter_theta} \\
e_\theta &= \hat{\theta}_{accel}-\hat{\theta}, \notag
\end{align}
\begin{align}
\dot{\hat{\psi}} &= (r_{gyro}-\hat{b}_\psi) + k_p e_\psi \notag \\
\dot{\hat{b}}_\psi &= -k_i e_\psi \label{eq:complementary_filter_psi} \\
e_\psi &= \hat{\psi}_{mag}-\hat{\psi}, \notag
\end{align}
where $\hat{\psi}_{mag}$ is the measurement of the yaw angle from a magnetometer.

## Nonlinear Complementary Filter for Attitude Kinematics

Consider the dynamics
\begin{align*}
\dot{R} &= R\omega^\wedge \\
\dot{\omega} &= -\frac{b}{I}\omega + \frac{1}{I}\tau.
\end{align*}
We will assume that the angular velocity $\omega$ is measured with some unknown bias $b$.  Therefore, the measurement is 
$$
\omega_m = \omega + b.
$$

For the complementary filter, we assume that we measure $\omega_m$ and also the attitude $R_m$.  Let $\bar{R}$, $\bar{\omega}$, and $\bar{b}$ be the estimate of $R$, $\omega$, and $b$.  

The passive complementary filter given in (Mahony, et. al, 2008) is given by
\begin{align*}
\dot{\bar{R}} &= \bar{R}\left(\omega_m-\bar{b}+k_pe_\Omega\right)^\wedge \\
\dot{\bar{b}} &= -k_I \Omega \\
e_\Omega &= \mathbb{P}_a(\bar{R}^\top R_y)^\vee.
\end{align*}

Proof of convergence is as follows.  
Define the Lyapunov function candidate as
\begin{align*}
V &= \frac{1}{4}\|I-\bar{R}^\top R\| + \frac{1}{2k_I}(b-\bar{b})^\top(b-\bar{b}) \\
&= \frac{1}{2}tr\left[I-\bar{R}^\top R\right] + \frac{1}{2k_I}(b-\bar{b})^\top(b-\bar{b}).
\end{align*}
Taking the derivative with respect to time gives
\begin{align*}
\dot{V} &= -\frac{1}{2}tr\left[\dot{\bar{R}}^\top R + \bar{R}^\top \dot{R}\right] - \frac{1}{k_I}(b-\bar{b})^\top \dot{\bar{b}}\\
&= -\frac{1}{2}tr\left[\left( \bar{R}\left(\omega_m-\bar{b}+k_pe_\Omega\right)^\wedge \right)^\top R + \bar{R}^\top R\omega^\wedge\right] + (b-\bar{b})^\top e_\Omega \\
&= -\frac{1}{2}tr\left[\left( \left(\omega_m-\bar{b}+k_pe_\Omega\right)^\wedge \right)^\top \bar{R}^\top R + \bar{R}^\top R\omega^\wedge\right] + (b-\bar{b})^\top e_\Omega \\
&= -\frac{1}{2}tr\left[-\left(\omega_m-\bar{b}+k_pe_\Omega\right)^\wedge \bar{R}^\top R + \bar{R}^\top R\omega^\wedge\right] + (b-\bar{b})^\top e_\Omega \\
&= -\frac{1}{2}tr\left[\bar{R}^\top R\omega^\wedge
- \bar{R}^\top R\omega_m^\wedge
+ \bar{R}^\top R\bar{b}^\wedge
- k_p \bar{R}^\top Re_\Omega^\wedge
\right] + (b-\bar{b})^\top e_\Omega \\
&= -\frac{1}{2}tr\left[\bar{R}^\top R(\omega_m-b)^\wedge
- \bar{R}^\top R\omega_m^\wedge
+ \bar{R}^\top R\bar{b}^\wedge
- k_p \bar{R}^\top Re_\Omega^\wedge
\right] + (b-\bar{b})^\top e_\Omega \\
&= \frac{1}{2}tr\left[\bar{R}^\top R(b-\bar{b})^\wedge
+ k_p \bar{R}^\top Re_\Omega^\wedge
\right] + (b-\bar{b})^\top e_\Omega \\
&= \frac{1}{2}tr\left[(\mathbb{P}_s(\bar{R}^\top R)+\mathbb{P}_a(\bar{R}^\top R))(b-\bar{b})^\wedge
+ k_p (\mathbb{P}_s(\bar{R}^\top R)+\mathbb{P}_a(\bar{R}^\top R))e_\Omega^\wedge
\right] + (b-\bar{b})^\top e_\Omega \\
&= \frac{1}{2}tr\left[\mathbb{P}_a(\bar{R}^\top R)(b-\bar{b})^\wedge
+ k_p \mathbb{P}_a(\bar{R}^\top R)e_\Omega^\wedge
\right] + (b-\bar{b})^\top e_\Omega \\
&= \frac{1}{2}tr\left[e_\Omega^\wedge(b-\bar{b})^\wedge
+ k_p e_\Omega^\wedge e_\Omega^\wedge
\right] + (b-\bar{b})^\top e_\Omega \\
&= \frac{1}{2}tr\left[(b-\bar{b})^\wedge e_\Omega^\wedge
+ k_p e_\Omega^\wedge e_\Omega^\wedge
\right] + (b-\bar{b})^\top e_\Omega \\
&= -(b-\bar{b})^\top e_\Omega
- k_p e_\Omega^\top e_\Omega + (b-\bar{b})^\top e_\Omega \\
&= -k_p e_\Omega^\top e_\Omega
\end{align*}
where we have assumed that $R_y=R$.  Since $e_\Omega=0$ when $\bar{R}=R$, $\dot{V}$ is negative semi-definite.  Barbalat's lemma can be used to show that the system converges everywhere except for a set of measure zero where $\bar{R}^\top R = I$.